In [1]:
import pandas as pd
import numpy as np

datadf = pd.read_csv('data/visual-search-study-data-live-analyze.csv')
datadf['source'] = 'data'

dataMetricsdf = pd.read_csv('data/metrics.csv')
dataMetricsdf['source'] = 'data'

# get rid of outliers 
datadf = datadf.loc[datadf['timetaken'] < 10000]
datadf = datadf.loc[datadf['timetaken'] > 250]
datadf = datadf.loc[datadf['uuid'] != '']

basemaps = [
    'dark',
    'streets',
    'imagery'
]

# # add random all data to all other basemaps - same map so would have been same numbers 
# for baseMapIndex in range(len(basemaps)):
#     datadfRandomAll = datadf.loc[datadf['distractor'] == 'random']
#     datadfRandomAll = datadfRandomAll.loc[datadf['distractorlevel'] == 'all']
#     datadfRandomAll = datadfRandomAll.loc[datadf['basemap'] == 'none']
#     datadfRandomAll['source'] = 'copied'
#     datadfRandomAll['basemap'] = basemaps[baseMapIndex]
#     datadfRandomAll['distractorcat'] =  datadfRandomAll['distractorcat'].replace('none' , basemaps[baseMapIndex], regex=True)
#     datadf = datadf.append(datadfRandomAll, ignore_index=True)

# # add clustered mixed data to all other basemaps - same map so would have been same numbers 
# for baseMapIndex in range(len(basemaps)):
#     datadfRandomAll = datadf.loc[datadf['distractor'] == 'clustered']
#     datadfRandomAll = datadfRandomAll.loc[datadf['color'] == 'mixed']
#     datadfRandomAll = datadfRandomAll.loc[datadf['basemap'] == 'none']
#     datadfRandomAll['source'] = 'copied'
#     datadfRandomAll['basemap'] = basemaps[baseMapIndex]
#     datadfRandomAll['distractorcat'] =  datadfRandomAll['distractorcat'].replace('none' , basemaps[baseMapIndex], regex=True)
#     datadf = datadf.append(datadfRandomAll, ignore_index=True)
    
# create categories
datadf['grpcat'] = datadf['target'] + "-" + datadf['distractorcat'] 
datadf['category'] = datadf['distractorcat'] 
datadf['posthoccat'] = datadf['distractor'] + '-' + datadf['distractorlevel'] + '-' + datadf['color'] + '-' + datadf['target'].replace(' ' ,'_', regex=True)
datadf['posthoccat2'] = datadf['distractor'] + '-' + datadf['distractorlevel'] + '-' + datadf['color']
datadf['posthoccat3'] = datadf['distractor'] + '-' + datadf['color'] + '-' + datadf['target'].replace(' ' ,'_', regex=True)

# log metrics
dataMetricsdf['noplog'] = np.log(dataMetricsdf['number_of_patches'])
dataMetricsdf['contagionlog'] = np.log(dataMetricsdf['contagion'])
dataMetricsdf['shapeindexlog'] = np.log(dataMetricsdf['shape_index_mn'])
dataMetricsdf['fracdimlog'] = np.log(dataMetricsdf['fractal_dimension_mn'])
dataMetricsdf['ennlog'] = np.log(dataMetricsdf['euclidean_nearest_neighbor_mn'])
dataMetricsdf['patchensitylog'] = np.log(dataMetricsdf['patch_density'])
dataMetricsdf['areamnlog'] = np.log(dataMetricsdf['area_mn'])

# merge metrics and data
datadf =  pd.merge(datadf, dataMetricsdf, on="distractorcat", how="left")

# export merged data
datadf.to_csv('data/data-and-metrics.csv')

/Users/daveism/.pyenv/versions/3.8.6/lib/python3.8/site-packages/pandas/core/arraylike.py:274: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [2]:
datadf['searchmapurl']

0         target-not-gestalt-23-distractor-random-high-d...
1         target-not-gestalt-19-distractor-random-med-di...
2         target-not-gestalt-36-distractor-random-high-d...
3         target-not-gestalt-28-distractor-random-low-di...
4         target-none-distractor-random-med-distractor-c...
                                ...                        
109722    target-not-gestalt-38-distractor-random-high-d...
109723    target-none-distractor-random-med-distractor-c...
109724    target-none-distractor-random-low-distractor-c...
109725        target-none-no-distractor-basemap-streets.png
109726    target-none-distractor-random-med-distractor-c...
Name: searchmapurl, Length: 109727, dtype: object

In [147]:
import matplotlib.pyplot as plt
import seaborn as sns


# basemap = 'none'
highlim = 10

metric = 'number_of_patches'
datadf['nop_rank'] = pd.qcut(datadf[metric], highlim, labels = False)
metric = 'euclidean_nearest_neighbor_mn'
datadf['enn_rank'] = pd.qcut(datadf[metric], highlim, labels = False)
metric = 'contagion'
datadf['cont_rank'] = pd.qcut(datadf[metric], highlim, labels = False)
datadf

datadfMetricRankPivot = pd.pivot_table(datadf, values=['timetaken'], index=['enn_rank'], columns=['target'], aggfunc=np.mean)
datadfMetricRankPivot = datadfMetricRankPivot.rename_axis(None, axis=0)
datadfMetricRankPivot.columns = datadfMetricRankPivot.columns.get_level_values(1)
datadfMetricRankPivot['diff'] = datadfMetricRankPivot['less gestalt'] - datadfMetricRankPivot['gestalt'] 
datadfMetricRankPivot = datadfMetricRankPivot.sort_values(["diff"], ascending = (False))
datadfMetricRankPivot['rank'] = datadfMetricRankPivot.index
datadfMetricRankPivot
# # sns.lineplot(data=datadfMetricRankPivot, x="rank", y="diff")

# val = datadfMetricRankPivot.index.values[0]

# datadfLim = datadf.loc[datadf['cont_rank'] == val]
# grouped_single = datadfLim.groupby(['file', 'target']).agg({'timetaken': ['mean']})
# grouped_single = grouped_single.reset_index()
# grouped_single.columns = ['file', 'target', 'timetaken_mean']
# grouped_single


# Pivot = pd.pivot_table(grouped_single, values=['timetaken_mean'], index=['file'], columns=['target'], aggfunc=np.mean)
# Pivot = Pivot.rename_axis(None, axis=0)
# Pivot.columns = Pivot.columns.get_level_values(1)
# Pivot['file'] = Pivot.index
# Pivot['file'] = Pivot['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/target-none/', '', regex=True)
# Pivot['file'] = Pivot['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/target-none/', '', regex=True)
# Pivot['file'] = Pivot['file'].replace('/tiledata/maps/missed/', '', regex=True)
# Pivot['file'] = Pivot['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/blue-target-none/', '', regex=True) 
# Pivot['file'] = Pivot['file'].replace('.tif', '.png', regex=True) 
# Pivot['diff'] = Pivot['less gestalt'] - Pivot['gestalt'] 
# Pivot = Pivot.sort_values(["diff"], ascending = (False))
# Pivot = Pivot.reset_index()
# Pivot.columns = ['rem', 'gestalt', 'less gestalt', 'no target', 'file', 'diff']
# Pivot = Pivot.drop(columns=['rem','gestalt', 'less gestalt', 'no target'])
# Pivot['rank'] = val
# Pivot

# Pivot.to_csv('data/top_maps-enn_rank-baddata.csv')




target,gestalt,less gestalt,no target,diff,rank
0,1361.078246,1677.225857,1823.472983,316.147611,0
8,1433.778866,1651.620631,1846.150918,217.841766,8
6,1445.037750,1642.797496,1837.270202,197.759747,6
7,1419.032154,1604.935720,1800.978750,185.903565,7
5,1554.901832,1738.716878,1993.304838,183.815047,5
4,1513.660637,1679.143697,1894.401937,165.483060,4
1,1397.522459,1553.198223,1779.274450,155.675764,1
2,1316.845004,1415.449122,1645.476470,98.604118,2
9,1266.453763,1352.363557,1571.501933,85.909794,9
3,1403.233804,1441.697947,1629.148405,38.464143,3


In [184]:
highlim = 10

metric = 'number_of_patches'
datadf['nop_rank'] = pd.qcut(datadf[metric], highlim, labels = False)

metric = 'euclidean_nearest_neighbor_mn'
datadf['enn_rank'] = pd.qcut(datadf[metric], highlim, labels = False)

metric = 'contagion'
datadf['cont_rank'] = pd.qcut(datadf[metric], highlim, labels = False)



datadfsort = datadf.loc[datadf['target'] != 'no target']
datadfsort = datadfsort.reset_index()
datadfsort = datadfsort.sort_values(["timetaken","contagion"], ascending = (False))
datadfsort['file'] = datadfsort['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/target-none/', '', regex=True)
datadfsort['file'] = datadfsort['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/target-none/', '', regex=True)
datadfsort['file'] = datadfsort['file'].replace('/tiledata/maps/missed/', '', regex=True)
datadfsort['file'] = datadfsort['file'].replace('/tiledata/maps/map-for-s3-tifs-georef/blue-target-none/', '', regex=True) 
datadfsort['file'] = datadfsort['file'].replace('.tif', '.png', regex=True) 
grouped_single = datadfsort.groupby(['file', 'target']).agg({'timetaken': ['mean'],
                                                             'nop_rank': ['mean'],
                                                             'enn_rank': ['mean'],
                                                             'cont_rank': ['mean']})
grouped_single = grouped_single.reset_index()
grouped_single.columns = ['file', 'target', 'timetaken_mean', 'nop_mean', 'enn_mean', 'cont_mean']

Pivot = (grouped_single.pivot_table(index=['file','nop_mean', 'enn_mean', 'cont_mean'],
                      columns='target', values='timetaken_mean')
       .reset_index()
       .rename_axis(None, axis=1))

Pivot['diff'] = Pivot['less gestalt'] - Pivot['gestalt'] 
Pivot['diffpercent'] = Pivot['diff']/Pivot['gestalt'] 
Pivot = Pivot.sort_values(["diffpercent"], ascending = (False))
Pivot
Pivot.to_csv('data/biggest_differences.csv')


Pivot = Pivot.sort_values(["less gestalt"], ascending = (False))
Pivot
Pivot.to_csv('data/rt_highest_lessgestalt.csv')

Pivot = Pivot.sort_values(["gestalt"], ascending = (False))
Pivot
Pivot.to_csv('data/rt_highest_gestalt.csv')
